목표
--------
* url을 각각의 변수로 나누어서 실습한다.
* erd를 보고 필요한 내용들을 파싱해본다.
* 파싱한 데이터를 db에 저장한다.

In [13]:
#필요한 라이브러리 import
import requests
import urllib.request
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pymysql

In [3]:
conn = pymysql.connect(host="localhost", user='queant', password='queant1234', db = 'queant', charset='utf8mb4')
cur = conn.cursor()

In [40]:
#api활용할 url부분

#각 데이터 별 url 앞부분
url_deposit = ["https://finlife.fss.or.kr/finlifeapi/depositProductsSearch.xml?auth=", "https://finlife.fss.or.kr/finlifeapi/savingProductsSearch.xml?auth="]

#발급받은 키
api_key = "47c0e868fdb16333d47d0e385641c3c0"

#권역코드
code_front = "&topFinGrpNo="
code_bank = ["020000", "030300"]

#페이지
page_front = "&pageNo="
page_num = ["1", "2", "3", "4"]

In [41]:
url = url_deposit[0] + api_key + code_front + code_bank[1] + page_front + page_num[0]
print(url)
data_str = urllib.request.urlopen(url).read().decode('euc-kr')
data_xml = ET.fromstring(data_str)

https://finlife.fss.or.kr/finlifeapi/depositProductsSearch.xml?auth=47c0e868fdb16333d47d0e385641c3c0&topFinGrpNo=030300&pageNo=1


* 아래 코드는 예,적금 상품의 가입방법을 파싱하는 코드
    * join_way 태그에 접근해서 데이터를 받는다.
    * 기존의 데이터는 스마트폰, 인터넷, 영업점 등으로 표현되어있다.
    * DB에는 공통코드를 통해 저장할 것이기 때문에 각각의 가입방법 존재유무를 파악해야한다.
    * Api를 볼때 가입방법은 총 영업점, 인터넷, 스마트폰, 전화, 기타로 총 5가지이다.
    * 각각의 문자열이 join_way 데이터에 있는지 확인하고 그 유무를 True, False로 배열에 저장할 것.

In [57]:
for product_tag in data_xml[5]:
    join = product_tag[0].find("join_way").text
    print(join)
    join_parsing = ["영업점" in join, "인터넷" in join, "스마트폰" in join, "전화" in join, "기타" in join]
    print(join_parsing)
    print()

영업점,인터넷,스마트폰,전화(텔레뱅킹)
[True, True, True, True, False]

영업점,스마트폰
[True, False, True, False, False]

스마트폰
[False, False, True, False, False]

영업점,인터넷,스마트폰
[True, True, True, False, False]

스마트폰
[False, False, True, False, False]

영업점
[True, False, False, False, False]

영업점,인터넷,스마트폰
[True, True, True, False, False]

인터넷,스마트폰
[False, True, True, False, False]

영업점,인터넷,스마트폰
[True, True, True, False, False]

영업점,인터넷,스마트폰
[True, True, True, False, False]

스마트폰
[False, False, True, False, False]

영업점,인터넷,스마트폰
[True, True, True, False, False]

스마트폰
[False, False, True, False, False]

영업점,인터넷,스마트폰
[True, True, True, False, False]

인터넷
[False, True, False, False, False]

스마트폰
[False, False, True, False, False]

영업점,인터넷,스마트폰
[True, True, True, False, False]

인터넷,스마트폰
[False, True, True, False, False]

인터넷,스마트폰
[False, True, True, False, False]

영업점
[True, False, False, False, False]

인터넷
[False, True, False, False, False]

스마트폰
[False, False, True, False, False]

스마트폰
[False, False, True, False, F

* 아래 코드는 예,적금 상품의 가입 제한을 파싱하는 코드
    * 데이터들을 보았을때 일정한 형식이 없다.
    * 이를 어떻게 파싱할지 계획하는 것이 중요.

In [42]:
for product_tag in data_xml[5]:
    join_deny = product_tag[0].find("join_deny").text
    join_member = product_tag[0].find("join_member").text
    print(join_deny, join_member)

1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 개인고객
1 개인고객
1 누구나 가입가능
1 누구나가입가능
1 누구나 가입가능
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 개인
1 개인
1 개인
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 만 17세이상
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 만 19세 이상의 개인
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 -제한없음
1 -만19세이상의 개인
1 -제한없음
1 -만19세이상의 개인
1 제한없음
1 인터넷 및 모바일뱅킹 사용자
1 비대면 어플사용자
1 제한없음
1 인터넷 및 모바일뱅킹 사용자
1 누구나가입가능
1 누구나가입가능
1 인터넷뱅킹가입 대상
1 인터넷뱅킹가입 대상
1 제한없음
1 제한 없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 인터넷뱅킹 및 스마트폰 뱅킹 신청자
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 제한없음
1 만19세이상 내국인 개인고객, 본인 스마트폰 소지자, 신분증(주민등록증 또는 운전면허증)소지자, 미국인 납세자에 해당하지 않는 고객


* 아래 코드는 가입 최소금액, 최대금액을 파싱할 코드
    * 어떻게 데이터를 뽑아내지...?

In [ ]:
for product_tag in data_xml[5]:
    etc = product_tag[0].find("etc_note").text
    print(etc)
    print()